In [3]:
class arguments(object):
    def __init__(self, df_name, S, Y, underprivileged_value, desirable_value, num_epochs, batch_size, lambda_val, fake_name, size_of_fake_data):
        self.df_name = df_name
        self.S = S
        self.Y = Y
        self.underprivileged_value = underprivileged_value
        self.desirable_value = desirable_value
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.lambda_val = lambda_val
        self.fake_name = fake_name
        self.size_of_fake_data = size_of_fake_data

In [4]:
args = arguments("Compas_Raw_Scores_Modified.csv", "Ethnic_Code_Text", "binary_text", "African-American", "Low_Chance", 100, 256, 1.0 , "fake_compas.csv", 16267)

In [5]:
import torch
import torch.nn.functional as f
from torch import nn
import pandas as pd
import numpy as np
import torch
from collections import OrderedDict

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt



"""
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("df_name", help="Reference dataframe", type=str)
parser.add_argument("S", help="Protected attribute", type=str)
parser.add_argument("Y", help="Label (decision)", type=str)
parser.add_argument("underprivileged_value", help="Value for underpriviledged group", type=str)
parser.add_argument("desirable_value", help="Desired label (decision)", type=str)

parser.add_argument("num_epochs", help="Total number of epochs", type=int)
parser.add_argument("batch_size", help="the batch size", type=int)
#hilbert change
#parser.add_argument("num_fair_epochs", help="number of fair training epochs", type=int)
parser.add_argument("lambda_val", help="lambda parameter", type=float)
parser.add_argument("fake_name", help="name of the produced csv file", type=str)
parser.add_argument("size_of_fake_data", help="how many data records to generate", type=int)
args = parser.parse_args()
"""


S = args.S
Y = args.Y
S_under = args.underprivileged_value
Y_desire = args.desirable_value

df = pd.read_csv(args.df_name)

df[S] = df[S].astype(object)
df[Y] = df[Y].astype(object)







def get_ohe_data(df):
    df_int = df.select_dtypes(['float', 'integer']).values
    continuous_columns_list = list(df.select_dtypes(['float', 'integer']).columns)
    ##############################################################
    scaler = QuantileTransformer(n_quantiles=2000, output_distribution='uniform')
    df_int = scaler.fit_transform(df_int)

    df_cat = df.select_dtypes('object')
    df_cat_names = list(df.select_dtypes('object').columns)
    numerical_array = df_int
    ohe = OneHotEncoder()
    ohe_array = ohe.fit_transform(df_cat)

    cat_lens = [i.shape[0] for i in ohe.categories_]
    discrete_columns_ordereddict = OrderedDict(zip(df_cat_names, cat_lens))

    S_start_index = len(continuous_columns_list) + sum(
        list(discrete_columns_ordereddict.values())[:list(discrete_columns_ordereddict.keys()).index(S)])
    Y_start_index = len(continuous_columns_list) + sum(
        list(discrete_columns_ordereddict.values())[:list(discrete_columns_ordereddict.keys()).index(Y)])

    if ohe.categories_[list(discrete_columns_ordereddict.keys()).index(S)][0] == S_under:
        underpriv_index = 0
        priv_index = 1
    else:
        underpriv_index = 1
        priv_index = 0
    if ohe.categories_[list(discrete_columns_ordereddict.keys()).index(Y)][0] == Y_desire:
        desire_index = 0
        undesire_index = 1
    else:
        desire_index = 1
        undesire_index = 0

    final_array = np.hstack((numerical_array, ohe_array.toarray()))
    return ohe, scaler, discrete_columns_ordereddict, continuous_columns_list, final_array, S_start_index, Y_start_index, underpriv_index, priv_index, undesire_index, desire_index


def get_original_data(df_transformed, df_orig, ohe, scaler):
    # df_int = df_orig.select_dtypes(['float','integer'])
    df_ohe_int = df_transformed[:, :df_orig.select_dtypes(['float', 'integer']).shape[1]]
    df_ohe_int = scaler.inverse_transform(df_ohe_int)
    df_ohe_cats = df_transformed[:, df_orig.select_dtypes(['float', 'integer']).shape[1]:]
    df_ohe_cats = ohe.inverse_transform(df_ohe_cats)
    # df_income = df_transformed[:,-1]
    # df_ohe_cats = np.hstack((df_ohe_cats, df_income.reshape(-1,1)))
    df_int = pd.DataFrame(df_ohe_int, columns=df_orig.select_dtypes(['float', 'integer']).columns)
    df_cat = pd.DataFrame(df_ohe_cats, columns=df_orig.select_dtypes('object').columns)
    return pd.concat([df_int, df_cat], axis=1)








def prepare_data(df, batch_size):
    #df = pd.concat([df_train, df_test], axis=0)

    ohe, scaler, discrete_columns, continuous_columns, df_transformed, S_start_index, Y_start_index, underpriv_index, priv_index, undesire_index, desire_index = get_ohe_data(df)


    input_dim = df_transformed.shape[1]

    #from sklearn.model_selection import train_test_split
    #################
    X_train, X_test = train_test_split(df_transformed,test_size=0.1, shuffle=True) #random_state=10)
    #X_train = df_transformed[:df_train.shape[0],:]
    #X_test = df_transformed[df_train.shape[0]:,:]

    data_train = X_train.copy()
    data_test = X_test.copy()

    from torch.utils.data import TensorDataset
    from torch.utils.data import DataLoader
    data = torch.from_numpy(data_train).float()


    train_ds = TensorDataset(data)
    train_dl = DataLoader(train_ds, batch_size = batch_size, drop_last=True)
    return ohe, scaler, input_dim, discrete_columns, continuous_columns ,train_dl, data_train, data_test, S_start_index, Y_start_index, underpriv_index, priv_index, undesire_index, desire_index


class Generator(nn.Module):
    def __init__(self, input_dim, continuous_columns, discrete_columns):
        super(Generator, self).__init__()
        self._input_dim = input_dim
        self._discrete_columns = discrete_columns
        self._num_continuous_columns = len(continuous_columns)

        self.lin1 = nn.Linear(self._input_dim, self._input_dim)
        self.lin_numerical = nn.Linear(self._input_dim, self._num_continuous_columns)

        self.lin_cat = nn.ModuleDict()
        for key, value in self._discrete_columns.items():
            self.lin_cat[key] = nn.Linear(self._input_dim, value)

    def forward(self, x):
        x = torch.relu(self.lin1(x))
        # x = f.leaky_relu(self.lin1(x))
        # x_numerical = f.leaky_relu(self.lin_numerical(x))
        x_numerical = f.relu(self.lin_numerical(x))
        x_cat = []
        for key in self.lin_cat:
            x_cat.append(f.gumbel_softmax(self.lin_cat[key](x), tau=0.2))
        x_final = torch.cat((x_numerical, *x_cat), 1)
        return x_final


class Critic(nn.Module):
    def __init__(self, input_dim):
        super(Critic, self).__init__()
        self._input_dim = input_dim
        # self.dense1 = nn.Linear(109, 256)
        self.dense1 = nn.Linear(self._input_dim, self._input_dim)
        self.dense2 = nn.Linear(self._input_dim, self._input_dim)
        # self.dense3 = nn.Linear(256, 1)
        # self.drop = nn.Dropout(p=0.2)
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = f.leaky_relu(self.dense1(x))
        # x = self.drop(x)
        # x = f.leaky_relu(self.dense2(x))
        x = f.leaky_relu(self.dense2(x))
        # x = self.drop(x)
        return x

    
    
    
def pairwise_distances(x):
    #x should be two dimensional
    instances_norm = torch.sum(x**2,-1).reshape((-1,1))
    return -2*torch.mm(x,x.t()) + instances_norm + instances_norm.t()

def GaussianKernelMatrix(x, sigma=1):
    if sigma == 0:
        sigma = 0.0001
    pairwise_distances_ = pairwise_distances(x)
    return torch.exp(-pairwise_distances_ /sigma)

def HSIC(X, Y):
    n = X.shape[0]
    Xmed = X
    G = torch.sum(Xmed*Xmed, 1).reshape(n,1)
    Q = torch.tile(G, (1,n))
    R = torch.tile(G.T, (n,1))
    
    dists = Q + R - 2*torch.matmul(Xmed, Xmed.T)
    dists = dists - torch.tril(dists)
    dists = dists.reshape(n**2, 1)
    
    if not dists[dists>0].nelement() == 0:
        s_x = torch.sqrt( 0.5 * torch.median(dists[dists>0]) )
    else:
        s_x = 0

    n = Y.shape[0]
    Ymed = Y
    G = torch.sum(Ymed*Ymed, 1).reshape(n,1)
    Q = torch.tile(G, (1,n))
    R = torch.tile(G.T, (n,1))
    
    dists = Q + R - 2*torch.matmul(Ymed, Ymed.T)
    dists = dists - torch.tril(dists)
    dists = dists.reshape(n**2, 1)
    
    if not dists[dists>0].nelement() == 0:
        s_y = torch.sqrt( 0.5 * torch.median(dists[dists>0]) )
    else:
        s_y = 0
    
    m,_ = X.shape #batch size
    K = GaussianKernelMatrix(X,s_x)
    L = GaussianKernelMatrix(Y,s_y)
    H = torch.eye(m) - 1.0/m * torch.ones((m,m))
    H = H.float().cuda()
    HSIC = torch.trace(torch.mm(L,torch.mm(H,torch.mm(K,H))))/((m-1)**2)
    return HSIC
    
   


    
class FairLossFunc(nn.Module):
    def __init__(self, S_start_index, Y_start_index, underpriv_index, priv_index, undesire_index, desire_index):
        super(FairLossFunc, self).__init__()
        self._S_start_index = S_start_index
        self._Y_start_index = Y_start_index
        self._underpriv_index = underpriv_index
        self._priv_index = priv_index
        self._undesire_index = undesire_index
        self._desire_index = desire_index

    def forward(self, x, crit_fake_pred, lamda):
        #G = x[:, self._S_start_index:self._S_start_index + 2]
        G = x[:, self._S_start_index:self._S_start_index + 1]

        # print(x[0,64])
        #I = x[:, self._Y_start_index:self._Y_start_index + 2]
        I = x[:, self._Y_start_index:self._Y_start_index + 1]
        #print(G.shape)
        #print(I.shape)
        # disp = (torch.mean(G[:,1]*I[:,1])/(x[:,65].sum())) - (torch.mean(G[:,0]*I[:,0])/(x[:,64].sum()))
        # disp = -1.0 * torch.tanh(torch.mean(G[:,0]*I[:,1])/(x[:,64].sum()) - torch.mean(G[:,1]*I[:,1])/(x[:,65].sum()))
        # gen_loss = -1.0 * torch.mean(crit_fake_pred)
        
        """
        disp = -1.0 * lamda * (torch.mean(G[:, self._underpriv_index] * I[:, self._desire_index]) / (
            x[:, self._S_start_index + self._underpriv_index].sum()) - torch.mean(
            G[:, self._priv_index] * I[:, self._desire_index]) / (
                                   x[:, self._S_start_index + self._priv_index].sum())) - 1.0 * torch.mean(
            crit_fake_pred)
        
        """
        disp = 1.0 * lamda * HSIC(G,I) - 1.0 * torch.mean(crit_fake_pred)
        
        
        # print(disp)
        return disp





device = torch.device("cuda:0" if (torch.cuda.is_available() and 1 > 0) else "cpu")


def get_gradient(crit, real, fake, epsilon):
    mixed_data = real * epsilon + fake * (1 - epsilon)

    mixed_scores = crit(mixed_data)

    gradient = torch.autograd.grad(
        inputs=mixed_data,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    return gradient


def gradient_penalty(gradient):
    gradient = gradient.view(len(gradient), -1)
    gradient_norm = gradient.norm(2, dim=1)

    penalty = torch.mean((gradient_norm - 1) ** 2)
    return penalty


def get_gen_loss(crit_fake_pred):
    gen_loss = -1. * torch.mean(crit_fake_pred)

    return gen_loss


def get_crit_loss(crit_fake_pred, crit_real_pred, gp, c_lambda):
    crit_loss = torch.mean(crit_fake_pred) - torch.mean(crit_real_pred) + c_lambda * gp

    return crit_loss









display_step = 50


def train(df, epochs=500, batch_size=64, lamda=0.5):
    ohe, scaler, input_dim, discrete_columns, continuous_columns, train_dl, data_train, data_test, S_start_index, Y_start_index, underpriv_index, priv_index, undesire_index, desire_index = prepare_data(
        df, batch_size)

    generator = Generator(input_dim, continuous_columns, discrete_columns).to(device)
    critic = Critic(input_dim).to(device)
    second_critic = FairLossFunc(S_start_index, Y_start_index, underpriv_index, priv_index, undesire_index, desire_index).to(
        device)

    gen_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    gen_optimizer_fair = torch.optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.999))
    crit_optimizer = torch.optim.Adam(critic.parameters(), lr=0.0002, betas=(0.5, 0.999))

    # loss = nn.BCELoss()
    critic_losses = []
    generator_losses = []
    cur_step = 0
    for i in range(epochs):
        # j = 0
        #print("epoch {}".format(i + 1))
        ############################
        #hilbert change
        #if i + 1 <= (epochs - fair_epochs):
        #    print("training for accuracy")
        #if i + 1 > (epochs - fair_epochs):
        #    print("training for fairness")
        for data in train_dl:
            data[0] = data[0].to(device)
            # j += 1
            loss_of_epoch_G = 0
            loss_of_epoch_D = 0
            crit_repeat = 5
            mean_iteration_critic_loss = 0
            for k in range(crit_repeat):
                # training the critic
                crit_optimizer.zero_grad()
                fake_noise = torch.randn(size=(batch_size, input_dim), device=device).float()
                fake = generator(fake_noise)

                crit_fake_pred = critic(fake.detach())
                crit_real_pred = critic(data[0])

                epsilon = torch.rand(batch_size, input_dim, device=device, requires_grad=True)
                gradient = get_gradient(critic, data[0], fake.detach(), epsilon)
                gp = gradient_penalty(gradient)

                crit_loss = get_crit_loss(crit_fake_pred, crit_real_pred, gp, c_lambda=10)

                mean_iteration_critic_loss += crit_loss.item() / crit_repeat
                crit_loss.backward(retain_graph=True)
                crit_optimizer.step()
            #############################
            if cur_step > 50:
                critic_losses += [mean_iteration_critic_loss]

            #############################
            #hilbert change
            # training the generator for fairness
            # training the generator for accuracy


            gen_optimizer_fair.zero_grad()
            fake_noise_2 = torch.randn(size=(batch_size, input_dim), device=device).float()
            fake_2 = generator(fake_noise_2)
            crit_fake_pred_2 = critic(fake_2)
            gen_fair_loss = second_critic(fake_2, crit_fake_pred_2, lamda)
            gen_fair_loss.backward()
            gen_optimizer_fair.step()
            
            #hilbert change
            """
            if i + 1 <= (epochs - fair_epochs):
                # training the generator for accuracy
                gen_optimizer.zero_grad()
                fake_noise_2 = torch.randn(size=(batch_size, input_dim), device=device).float()
                fake_2 = generator(fake_noise_2)
                crit_fake_pred = critic(fake_2)

                gen_loss = get_gen_loss(crit_fake_pred)
                gen_loss.backward()

                # Update the weights
                gen_optimizer.step()

            ###############################
            if i + 1 > (epochs - fair_epochs):
                # training the generator for fairness
                gen_optimizer_fair.zero_grad()
                fake_noise_2 = torch.randn(size=(batch_size, input_dim), device=device).float()
                fake_2 = generator(fake_noise_2)
                crit_fake_pred = critic(fake_2)

                gen_fair_loss = second_critic(fake_2, crit_fake_pred, lamda)
                gen_fair_loss.backward()
                gen_optimizer_fair.step()


            """
            
            # Keep track of the average generator loss
            #################################
            if cur_step > 50:
                """
                if i + 1 <= (epochs - fair_epochs):
                    generator_losses += [gen_loss.item()]
                if i + 1 > (epochs - fair_epochs):
                    generator_losses += [gen_fair_loss.item()]
                """
                generator_losses += [gen_fair_loss.item()]

                
            """
            if cur_step % display_step == 0 and cur_step > 0:
                gen_mean = sum(generator_losses[-display_step:]) / display_step
                crit_mean = sum(critic_losses[-display_step:]) / display_step
                print("Step {}: Generator loss: {}, critic loss: {}".format(cur_step, gen_mean, crit_mean))
                step_bins = 20
                num_examples = (len(generator_losses) // step_bins) * step_bins
                #plt.ion()
                plt.plot(
                    range(num_examples // step_bins),
                    torch.Tensor(generator_losses[:num_examples]).view(-1, step_bins).mean(1),
                    label="Generator Loss"
                )
                plt.plot(
                    range(num_examples // step_bins),
                    torch.Tensor(critic_losses[:num_examples]).view(-1, step_bins).mean(1),
                    label="Critic Loss"
                )
                plt.legend()
                plt.show()
            """
                  # print("cr step: {}".format(cur_step))
            
            
                    
            cur_step += 1

    return generator, critic, ohe, scaler, data_train, data_test, input_dim







#generator, critic, ohe, scaler, data_train, data_test = train_plot(df, args.num_epochs,args.batch_size, args.num_fair_epochs, args.lambda_val)
def train_plot(df, epochs, batchsize, lamda):
    generator, critic, ohe, scaler, data_train, data_test, input_dim = train(df, epochs, batchsize, lamda)
    return generator, critic, ohe, scaler, data_train, data_test, input_dim


In [43]:
#df = pd.read_csv("adult.csv")
#generator, critic, ohe, scaler, data_train, data_test, input_dim = train_plot(df, 100, args.batch_size, 10.0)

In [7]:
outfile = "results.log"

def print2file(buf, outFile):
    outfd = open(outFile, 'a')
    outfd.write(buf + '\n')
    outfd.close()

firstline = "epoch, lambda, acc, f1, pmale, pfemale, dp, omale, ofemale, deo"
print2file(firstline, outfile)
#epoch_val = 100
for epoch_val in [100,150]:
#for epoch_val in np.linspace(1,5,5):
    #for lambda_val in np.linspace(0.05,0.15,11):
    for lambda_val in [1.0,1.2,1.4,1.6,1.8,2]:
        #for repetition in range(5):
        
        print("epoch: {}, lambda: {}".format(epoch_val, lambda_val))
        #print("epochs: {}, lambda: {}".format(int(epoch_val), lambda_val))
        #generator, critic, ohe, scaler, data_train, data_test, input_dim = train_plot(df, args.num_epochs, args.batch_size, args.num_fair_epochs, args.lambda_val)
        ##hilbert change
        df = pd.read_csv("Compas_Raw_Scores_Modified.csv")
        generator, critic, ohe, scaler, data_train, data_test, input_dim = train_plot(df, int(epoch_val), args.batch_size, lambda_val)
        
        df = pd.read_csv("Compas_Raw_Scores_Modified.csv")
        with torch.no_grad():
            fake_numpy_array = generator(torch.randn(size=(args.size_of_fake_data, input_dim), device=device)).cpu().detach().numpy()

        fake_df = get_original_data(fake_numpy_array, df, ohe, scaler)
        fake_df = fake_df[df.columns]
        fake_df.to_csv(args.fake_name, index=False)

        df_fake = pd.read_csv("fake_compas.csv")
        df_fake = pd.get_dummies(df_fake)

        df = pd.get_dummies(df)

        for i in df.columns:
            if df[i].dtype == "int64":
                df[i] = df[i].astype(float)

        for i in df.columns:
            if i not in df_fake:
                df_fake[i] = 0

        df_fake = df_fake[df.columns]


        df = df[df.columns.drop('Ethnic_Code_Text_African-American')]
        df = df[df.columns.drop('binary_text_High_Chance')]

        df_fake = df_fake[df_fake.columns.drop('Ethnic_Code_Text_African-American')]
        df_fake = df_fake[df_fake.columns.drop('binary_text_High_Chance')]


        from sklearn.model_selection import train_test_split
        a, df_test = train_test_split(df, test_size=0.2)
        df_train = df_fake
        df_train_x = df_train.loc[:,df.columns != 'binary_text_Low_Chance']
        df_train_y = df_train['binary_text_Low_Chance']
        df_test_x = df_test.loc[:,df.columns != 'binary_text_Low_Chance']
        df_test_y = df_test['binary_text_Low_Chance']

        #from sklearn.neural_network import MLPClassifier
        #clf = MLPClassifier()

        from sklearn.tree import DecisionTreeClassifier
        clf = DecisionTreeClassifier()

        clf = clf.fit(df_train_x, df_train_y)
        from sklearn.metrics import accuracy_score
        #print("accuracy  = {}".format(accuracy_score(clf.predict(df_test_x), df_test_y)))
        theaccuracy = accuracy_score(clf.predict(df_test_x), df_test_y)
        
        from sklearn.metrics import f1_score
        thef1 = f1_score(clf.predict(df_test_x), df_test_y)
        #print("f1  = {}".format(f1_score(clf.predict(df_test_x), df_test_y)))


        df_test_x['pred'] = clf.predict(df_test_x)
        df_test_x['real'] = df_test_y
        pmale = df_test_x[(df_test_x['Ethnic_Code_Text_Caucasian'] == 1) & (df_test_x['pred'] == 1)].shape[0] / df_test_x[df_test_x['Ethnic_Code_Text_Caucasian'] == 1].shape[0]
        pfemale = df_test_x[(df_test_x['Ethnic_Code_Text_Caucasian'] == 0) & (df_test_x['pred'] == 1)].shape[0] / df_test_x[df_test_x['Ethnic_Code_Text_Caucasian'] == 0].shape[0]
        DP = pmale - pfemale

        omale = df_test_x[(df_test_x['Ethnic_Code_Text_Caucasian'] == 1) & (df_test_x['pred'] == 1) & (df_test_x['real'] == 1)].shape[0] / df_test_x[(df_test_x['Ethnic_Code_Text_Caucasian'] == 1) & (df_test_x['real'] == 1)].shape[0]
        ofemale = df_test_x[(df_test_x['Ethnic_Code_Text_Caucasian'] == 0) & (df_test_x['pred'] == 1) & (df_test_x['real'] == 1)].shape[0] / df_test_x[(df_test_x['Ethnic_Code_Text_Caucasian'] == 0) & (df_test_x['real'] == 1)].shape[0]
        DiffEO = omale - ofemale


        the_line = "%d, %f, %f, %f, %f, %f, %f, %f, %f, %f" % (epoch_val, lambda_val, theaccuracy, thef1, pmale, pfemale, DP, omale, ofemale, DiffEO)
        print2file(the_line, outfile)
        

epoch: 100, lambda: 1.0
epoch: 100, lambda: 1.2
epoch: 100, lambda: 1.4
epoch: 100, lambda: 1.6
epoch: 100, lambda: 1.8
epoch: 100, lambda: 2
epoch: 150, lambda: 1.0
epoch: 150, lambda: 1.2
epoch: 150, lambda: 1.4
epoch: 150, lambda: 1.6
epoch: 150, lambda: 1.8
epoch: 150, lambda: 2


In [127]:
generator, critic, ohe, scaler, data_train, data_test, input_dim = train_plot(df, 100, args.batch_size, 0.09)

In [377]:
data_train_x = data_train[:, :-2]
data_train_y = np.argmax(data_train[:, -2:], axis=1)

data_test_x = data_test[:, :-2]
data_test_y = np.argmax(data_test[:, -2:], axis=1)

with torch.no_grad():
    df_generated = generator(torch.randn(size=(32561, input_dim), device=device)).cpu().detach().numpy()

df_generated_x = df_generated[:, :-2]
df_generated_y = np.argmax(df_generated[:, -2:], axis=1)

from sklearn.tree import DecisionTreeClassifier
#from sklearn.neural_network import MLPClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(data_train_x, data_train_y)
from sklearn.metrics import accuracy_score
print("accuracy original = {}".format(accuracy_score(clf.predict(data_test_x), data_test_y)))
#accuracy_original = accuracy_score(clf.predict(data_test_x), data_test_y)
from sklearn.metrics import f1_score
#f1_original = f1_score(clf.predict(data_test_x), data_test_y)
print("f1_original = {}".format(f1_score(clf.predict(data_test_x), data_test_y)))

from sklearn.tree import DecisionTreeClassifier
#from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(df_generated_x, df_generated_y)
from sklearn.metrics import accuracy_score
print("accuracy generated = {}".format(accuracy_score(clf.predict(data_test_x), data_test_y)))
#accuracy_generated = accuracy_score(clf.predict(data_test_x), data_test_y)
from sklearn.metrics import f1_score
#f1_generated = f1_score(clf.predict(data_test_x), data_test_y)
print("f1_generated = {}".format(f1_score(clf.predict(data_test_x), data_test_y)))

accuracy original = 0.8204708290685773
f1_original = 0.6279168434450573
accuracy generated = 0.7545547594677584
f1_generated = 0.5428898208158597


In [378]:
df = pd.read_csv("adult.csv")
with torch.no_grad():
    fake_numpy_array = generator(torch.randn(size=(args.size_of_fake_data, input_dim), device=device)).cpu().detach().numpy()

fake_df = get_original_data(fake_numpy_array, df, ohe, scaler)
fake_df = fake_df[df.columns]
fake_df.to_csv(args.fake_name, index=False)

df_fake = pd.read_csv("fake_adult.csv")
df_fake = pd.get_dummies(df_fake)

df = pd.get_dummies(df)

for i in df.columns:
    if df[i].dtype == "int64":
        df[i] = df[i].astype(float)

for i in df.columns:
    if i not in df_fake:
        df_fake[i] = 0

df_fake = df_fake[df.columns]

In [379]:
df = df[df.columns.drop('sex_ Female')]
df = df[df.columns.drop('income_ <=50K')]

df_fake = df_fake[df_fake.columns.drop('sex_ Female')]
df_fake = df_fake[df_fake.columns.drop('income_ <=50K')]


from sklearn.model_selection import train_test_split
a, df_test = train_test_split(df, test_size=0.2)
df_train = df_fake
df_train_x = df_train.loc[:,df.columns != 'income_ >50K']
df_train_y = df_train['income_ >50K']
df_test_x = df_test.loc[:,df.columns != 'income_ >50K']
df_test_y = df_test['income_ >50K']

#from sklearn.neural_network import MLPClassifier
#clf = MLPClassifier()

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

clf = clf.fit(df_train_x, df_train_y)
from sklearn.metrics import accuracy_score
print("accuracy  = {}".format(accuracy_score(clf.predict(df_test_x), df_test_y)))
#accuracy_original = accuracy_score(clf.predict(data_test_x), data_test_y)
from sklearn.metrics import f1_score
#f1_original = f1_score(clf.predict(data_test_x), data_test_y)
print("f1  = {}".format(f1_score(clf.predict(df_test_x), df_test_y)))


accuracy  = 0.757702937864674
f1  = 0.5417231364956439


In [380]:
df_test_x['pred'] = clf.predict(df_test_x)
df_test_x['real'] = df_test_y
print(df_test_x[(df_test_x['sex_ Male'] == 1) & (df_test_x['pred'] == 1)].shape[0] / df_test_x[df_test_x['sex_ Male'] == 1].shape[0])
print(df_test_x[(df_test_x['sex_ Male'] == 0) & (df_test_x['pred'] == 1)].shape[0] / df_test_x[df_test_x['sex_ Male'] == 0].shape[0])

0.299079754601227
0.2486919052016005


In [381]:
print(df_test_x[(df_test_x['sex_ Male'] == 1) & (df_test_x['pred'] == 1)].shape[0] / df_test_x[df_test_x['sex_ Male'] == 1].shape[0])

0.299079754601227


In [382]:
print(df_test_x[(df_test_x['sex_ Male'] == 0) & (df_test_x['pred'] == 1)].shape[0] / df_test_x[df_test_x['sex_ Male'] == 0].shape[0])

0.2486919052016005


In [343]:
##28 18
##55 62

In [383]:
print(df_test_x[(df_test_x['sex_ Male'] == 1) & (df_test_x['pred'] == 1) & (df_test_x['real'] == 1)].shape[0] / df_test_x[(df_test_x['sex_ Male'] == 1) & (df_test_x['real'] == 1)].shape[0])
print(df_test_x[(df_test_x['sex_ Male'] == 0) & (df_test_x['pred'] == 1) & (df_test_x['real'] == 1)].shape[0] / df_test_x[(df_test_x['sex_ Male'] == 0) & (df_test_x['real'] == 1)].shape[0])

0.5632126921170054
0.6743589743589744


In [345]:
df_test_x[(df_test_x['sex_ Male'] == 0) & (df_test_x['pred'] == 1) & (df_test_x['real'] == 1)].shape[0] / df_test_x[(df_test_x['sex_ Male'] == 0) & (df_test_x['real'] == 1)].shape[0]

0.6578171091445427

In [35]:
fake_numpy_array = generator(torch.randn(size=(args.size_of_fake_data, input_dim), device=device)).cpu().detach().numpy()
fake_df = get_original_data(fake_numpy_array, df, ohe, scaler)
fake_df = fake_df[df.columns]
fake_df.to_csv(args.fake_name, index=False)
####finish

In [36]:
import pandas as pd

In [37]:
fake = pd.read_csv("fake_adult.csv")

In [38]:
import table_evaluator

In [ ]:
import warnings
warnings.simplefilter("ignore")

from table_evaluator import load_data, TableEvaluator
table_evaluator = TableEvaluator(df, fake)
table_evaluator.visual_evaluation()